## `Actividades preliminares`

Comenzamos importando la libreria de pandas

In [102]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

Cargamos el dataset que utilizaran durante la investigacion

In [103]:
crime_scene_report     = pd.read_csv("./datasets/Crime Scene report.csv")
drivers_license        = pd.read_csv("./datasets/Drivers license.csv")
facebook_event_checkin = pd.read_csv("./datasets/facebook_event_checkin.csv")
getfitnow_checkin      = pd.read_csv("./datasets/GetFitNow check in.csv")
getfitnow_members      = pd.read_csv("./datasets/GetFitNow members.csv")
income                 = pd.read_csv("./datasets/income.csv")
interview              = pd.read_csv("./datasets/Interviews.csv")
person                 = pd.read_csv("./datasets/Person.csv")

Veamos un poco como estan conformados estos dataframes

In [104]:
# Usamos .info() para ver una descripcion general del dataset, como sus tipos de datos en cada columna y si presenta nulos
crime_scene_report.info()
#Podes cambiar el dataset para ver la informacion general del que quieras

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         1228 non-null   object
 1   Type         1228 non-null   object
 2   Description  939 non-null    object
 3   City         1228 non-null   object
dtypes: object(4)
memory usage: 38.5+ KB


In [105]:
crime_scene_report.head(3)

,Date,Type,Description,City
0,2017-01-07,smuggling,NaN,Savannah
1,2017-01-11,murder,NaN,Springdale
2,2017-01-12,smuggling,NaN,Melbourne


## `Comenzamos la investigacion`

### Un crimen ha ocurrido y el detective necesita tu ayuda. 


#### Pero antes, veamos un diagrama de esquema para ver como estan conectados todos los dataframes

<img src="./_src/schemeboard.png">


### Te entregaron el informe del lugar del crimen, pero de alguna manera lo perdiste. Recordas vagamente que el crimen fue un ​asesinato​ que ocurrió en algún momento el ​15 de enero de 2018​ y que tuvo lugar en ​Pandas City​. **Te recomiendo comenzar por recuperar el informe correspondiente de su respectivo dataframe**

In [106]:
informe = crime_scene_report[(crime_scene_report["Date"] == "2018-01-15") & 
                            (crime_scene_report["Type"] == "murder") &
                            (crime_scene_report["City"] == "Pandas City")]
informe

,Date,Type,Description,City
781,2018-01-15,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",Pandas City


In [107]:
#Reviso la descripcion
crime_scene_report['Description'][781]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [108]:
#Encuentro el primer testigo
person[person['Address_Street_Name'].str.contains('Northwestern Dr')].sort_values(by=['Address_Number']).tail()

,Id,Name,License_Id,Address_Number,Address_Street_Name,Ssn
9609,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
7040,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4886,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
811,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [109]:
#Encuentro al segundo testigo
person[person['Address_Street_Name'].str.contains('Franklin Ave') & person['Name'].str.contains('Annabel')] 

,Id,Name,License_Id,Address_Number,Address_Street_Name,Ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [110]:
#Entervista del primer testigo
interview[interview['Person_Id'] == 14887]['Transcript']


4988    I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
Name: Transcript, dtype: object

In [111]:
#Entreviste del segundo testigo
interview[interview['Person_Id'] == 16371]['Transcript']

4989    I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.
Name: Transcript, dtype: object

In [127]:
#Cambio el nombre de las columnas en drivers_licence y person, de esa manera hacerlo consistente
drivers_license=drivers_license.rename(columns={'Id':'License_Id'})
person = person.rename(columns={'Id':'Person_Id'})

#Creo una copia de getfitnow_members para no modificar la dataframe original
merged_df = getfitnow_members.copy()

#Con merge, combinamos las tablas para poder analizarlo en su totalidad
merged_df = merged_df.merge(person,on='Person_Id')
merged_df = merged_df.merge(drivers_license,on='License_Id')

In [113]:
#Se analiza la tabla con la informacion que encontramos
merged_df[merged_df['Id'].str.contains('48Z') &
          merged_df['Membership_Status'].str.contains('Gold') &
          merged_df['Plate_Number'].str.contains('H42W') &
          merged_df['Membership_Start_Date'].le('2018-01-09')
          ]

,Id,Person_Id,Name_x,Membership_Start_Date,Membership_Status,Name_y,License_Id,Address_Number,Address_Street_Name,Ssn,Age,Height,Eye_Color,Hair_Color,Gender,Plate_Number,Car_Make,Car_Model
65,48Z55,67318,Jeremy Bowers,2016-01-01,Gold,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,30,70,brown,Brown,Male,0H42W2,Chevrolet,Spark LS


## Si creen que encontraron al asesino, ejecuten el siguiente codigo con el posible sospechoso

 Por favor, tene en cuenta que es recomendable no acceder al archivo "asesino.py" para evitar revelar detalles importantes de la caso. Si deseas disfrutar plenamente de la experiencia sin spoilers, te sugiero que evites examinar dicho archivo."

In [114]:
from asesino import solucion
solucion("Jeremy Bowers")

¡Felicidades, encontraste al asesino! Pero espera, hay más...
 Si crees que estas preparado para un desafío, segui investigando 
 la transcripcion del asesino para encontrar al verdadero villano detrás de este crimen


In [115]:
interview[interview['Person_Id'] == 67318]['Transcript']

4990    I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.
Name: Transcript, dtype: object

In [133]:
merged_df2 = facebook_event_checkin.copy()
merged_df2 = merged_df2.rename(columns={'person_id':'Person_Id'})

merged_df2 = merged_df2.merge(person,on='Person_Id')
merged_df2 = merged_df2.merge(drivers_license,on='License_Id')


In [134]:
merged_df2[merged_df2['Height'].le(67) &
                merged_df2['Height'].ge(65) &
                merged_df2['Gender'].str.contains('Female') &
                merged_df2['Car_Make'].str.contains('Tesla') &
                merged_df2['Car_Model'].str.contains('Model S') &
                merged_df2['Hair_Color'].str.contains('Red') &
                merged_df2['event_name'].str.contains('SQL Symphony Concert')
                ]

,Person_Id,event_id,event_name,date,Name,License_Id,Address_Number,Address_Street_Name,Ssn,Age,Height,Eye_Color,Hair_Color,Gender,Plate_Number,Car_Make,Car_Model
20003,99716,1143,SQL Symphony Concert,20171206,Miranda Priestly,202298,1883,Golden Ave,987756388,68,66,green,Red,Female,500123,Tesla,Model S
20004,99716,1143,SQL Symphony Concert,20171212,Miranda Priestly,202298,1883,Golden Ave,987756388,68,66,green,Red,Female,500123,Tesla,Model S
20005,99716,1143,SQL Symphony Concert,20171229,Miranda Priestly,202298,1883,Golden Ave,987756388,68,66,green,Red,Female,500123,Tesla,Model S


In [119]:
solucion("Miranda Priestly")

¡Muy bien hecho! Encontraste a la mente detras del asesinato
 Todos en Pandas City te aclaman como el mejor detective de PANDAS de todos los tiempos. ¡Ya es hora de sacar el champagne!
